# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
import pickle
import random
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore') 

nltk.download(['words', 'punkt', 'stopwords', 'averaged_perceptron_tagger', 'maxent_ne_chunker', 'wordnet'])
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag, ne_chunk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer,CountVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,fbeta_score, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', con=engine)
df.head()

In [5]:
# Creating two separated dataframes
# X is the dataframe that contains the dependent column to be predicted : 'message'
X = df['message']
# Y is the dataframe that contains the independent columns (all binary)
Y = df.iloc[:,4:]
#Check the dimension of the X and Y dataframes
X.shape, Y.shape

((26028,), (26028, 36))

In [6]:
Y.head(3)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
X.head(3)

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
Name: message, dtype: object

### 2. Write a tokenization function to process your text data

In [8]:
def tokenize(text):
    #check for urls in the text
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text) 
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    # normalize the text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)

    #tokenize the text 
    tokens = word_tokenize(text)
    
    # Remove stop words
    tokens = [w for w in tokens if w not in stopwords.words("english")]
    
    #lemmatize the text
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

In [9]:
# Check to see if the tokenize function works properly
n = random.randint(0,len(X))
print('The original version:', X[n])
print('')
print('The tokenized version:', tokenize(X[n]))

The original version: This person would like to know who is responsible to clean the streets of the commune known as Carrefour. 

The tokenized version: ['this', 'person', 'would', 'like', 'know', 'responsible', 'clean', 'street', 'commune', 'known', 'carrefour']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=0)))
    ])
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fb53f7a4e18>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=0, ve

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
# Creation the train and test sets for the independent and dependent variables dataframes
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [12]:
# Display the dimension of all 4 dataframes
print('X_train shape: ', X_train.shape)
print('X_test shape: ', X_test.shape)
print('y_train shape: ', y_train.shape)
print('y_test shape: ', y_test.shape)

X_train shape:  (20822,)
X_test shape:  (5206,)
y_train shape:  (20822, 36)
y_test shape:  (5206, 36)


In [14]:
X_train.head(3)

6159     Hello, how are you, please I would like to kno...
327      they closed the bathroom and shower at the TNH...
22796    The Galkayo Water Project in the Northeast has...
Name: message, dtype: object

In [15]:
X_test.head(3)

7069     Good evening,How are you? Please, Excuse me. G...
10112          huge earthquake hits haiti http j.mp 8X5dq7
1559     I am in a very hard situation, I am a student ...
Name: message, dtype: object

In [16]:
y_train.head(3)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
6159,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
327,1,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
22796,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
y_test.head(3)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
7069,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10112,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
1559,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
#test the model 
y_pred = pipeline.predict(X_test)
print(y_pred)

In [21]:
# Source: https://stackoverflow.com/questions/56826216/valueerror-unknown-label-type-for-classification-report
#from sklearn.utils.multiclass import type_of_target
#type_of_target(y_test)
#type_of_target(y_pred)

'multilabel-indicator'

In [ ]:
print(classification_report(y_test, y_pred, target_names =Y.columns)) 

In [ ]:
accuracy = (y_pred == y_test).mean()
print("The calculated accuracy is :",accuracy)

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {
        'clf__estimator__n_estimators': [50, 100, 200],
        'clf__estimator__min_samples_split': [2, 3, 4]
    }
parameters

In [ ]:
cv = GridSearchCV(pipeline, param_grid=parameters)
cv

In [ ]:
cv.fit(X_train, y_train)

In [ ]:
y_pred = cv.predict(X_test)
Y_pred

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
print(classification_report(y_test, y_pred, target_names =Y.columns))

In [ ]:
accuracy = (y_pred == y_test).mean()
print("The calculated accuracy is :", accuracy)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
#Decision tree
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', DecisionTreeClassifier(()))
    ])
pipeline.get_params()

parameters = {
        'clf__estimator__n_estimators': [50, 100, 200],
        'clf__estimator__min_samples_split': [2, 3, 4]
    }
parameters

cv = GridSearchCV(pipeline, param_grid=parameters)
cv


cv.fit(X_train, y_train)


y_pred = cv.predict(X_test)
Y_pred

print(classification_report(y_test, y_pred, target_names =Y.columns))

accuracy = (y_pred == y_test).mean()
print("The calculated accuracy is :", accuracy)

In [ ]:
#KNN
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(KNeighborsClassifier()))
    ])
pipeline.get_params()

#Decision tree
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', DecisionTreeClassifier(()))
    ])
pipeline.get_params()

parameters = {
        'clf__estimator__n_estimators': [50, 100, 200],
        'clf__estimator__min_samples_split': [2, 3, 4]
    }
parameters

cv = GridSearchCV(pipeline, param_grid=parameters)
cv


cv.fit(X_train, y_train)


y_pred = cv.predict(X_test)
Y_pred

print(classification_report(y_test, y_pred, target_names =Y.columns))

accuracy = (y_pred == y_test).mean()
print("The calculated accuracy is :", accuracy)

### 9. Export your model as a pickle file

In [ ]:
#source:Grepper.com
# save the model to disk
filename = 'classifier.pkl'
pickle.dump(model, open(filename, 'wb'))

# load the model from disk
model = pickle.load(open(filename, 'rb'))
result = model.score(X_test, Y_test)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.